# Scrapper para obtener cédula profesional

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd

## Conexión al navegador

In [7]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://cedulaprofesionalsep.online/#Consulta_de_Cedula_Profesional'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

## Leer archivo CSV y obtener lista de nombres

In [2]:
# Cambiar por nombre del archivo
df = pd.read_csv('ejemplo.csv') 
#nombre = 'Bryan Antonio'
#paterno = 'Polito'
#materno = 'Palma'
df

,Nombre,Paterno,Materno
0,Bryan Antonio,Polito,Palma
1,Luis,Cruz,Jaramillo
2,Bryan Antonio,Polito,Palma
3,José Alberto,Montán,López


### Scrapper

In [58]:
driver.execute_script("window.scrollTo(10, 4000);")

In [8]:
data=[]
for index, row in df.iterrows():
    nombre=row['Nombre']
    paterno=row['Paterno']
    materno=row['Materno']
    # Ingresar nombres
    celda_nombre = driver.find_element(By.XPATH, '//*[@id="input-nombre"]')
    celda_nombre.send_keys(str(nombre))

    #Ingresar Apellido Paterno
    celda_paterno = driver.find_element(By.XPATH, '//*[@id="input-apaterno"]')
    celda_paterno.send_keys(str(paterno))

    # Ingresar Apellido Materno
    celda_materno = driver.find_element(By.XPATH, '//*[@id="input-amaterno"]')
    celda_materno.send_keys(str(materno))

    # Click en botón buscar
    buscar = driver.find_element(By.XPATH, '//*[@id="container-form-1"]/form/div[4]/div/input')
    buscar.click()
    time.sleep(2)

    max_wait_time = 5  # Tiempo máximo de espera en segundos
    start_time = time.time()  # Momento de inicio de espera

    while time.time() - start_time < max_wait_time:
        try:
            alert = driver.switch_to.alert
            alert.accept()
            driver.refresh()
            break  # Salir del bucle si se encuentra la ventana emergente
        except:

            time.sleep(1)  # Esperar 1 segundo antes de intentar nuevamente
            
            

    #tabla = driver.find_elements(By.XPATH, '//*[@id="result_table_row"]/tr[1]/td[3]/input')
        
        # Encontrar botones ver-mas
    ver_botones = driver.find_elements(By.CLASS_NAME, 'ver-mas')
        # Iterar sobre los botones
    
    
    for ver in ver_botones:
        cantidad_registros = len(ver_botones)
        print("Cantidad de registros:", cantidad_registros)

        #time.sleep()
        ver.click()
        
        # Encontrar los elementos de entrada de texto por su id
        input_cedula = driver.find_element(By.ID, "input-cedula-result")
        input_tipo = driver.find_element(By.ID, "input-tipo-result")
        input_sexo = driver.find_element(By.ID, "input-sexo-result")
        input_nombre = driver.find_element(By.ID, "input-nombre-result")
        input_paterno = driver.find_element(By.ID, "input-paterno-result")
        input_materno = driver.find_element(By.ID, "input-materno-result")
        input_escuela = driver.find_element(By.ID, "input-escuela-result")
        input_titulo = driver.find_element(By.ID, "input-titulo-result")

        # Obtener los valores de los campos
        valor_cedula = input_cedula.get_attribute("value")
        valor_tipo = input_tipo.get_attribute("value")
        valor_sexo = input_sexo.get_attribute("value")
        valor_nombre = input_nombre.get_attribute("value")
        valor_paterno = input_paterno.get_attribute("value")
        valor_materno = input_materno.get_attribute("value")
        valor_escuela = input_escuela.get_attribute("value")
        valor_titulo = input_titulo.get_attribute("value")

        # Agregar los valores a la lista de diccionarios
        data.append({
        "Cédula": valor_cedula,
        "Tipo de Cédula": valor_tipo,
        "Sexo": valor_sexo,
        "Nombre": valor_nombre,
        "Paterno": valor_paterno,
        "Materno": valor_materno,
        "Escuela": valor_escuela,
        "Título": valor_titulo
        })
        # Encontrar los elementos de entrada de texto por su id
        cerrar = driver.find_element(By.XPATH, '//*[@id="cerrar_resut_personal"]')
        cerrar.click()
    driver.refresh()

        
df_data = pd.DataFrame(data)
    

                

        

            


Cantidad de registros: 2
Cantidad de registros: 2
Cantidad de registros: 2
Cantidad de registros: 2


In [9]:
df_data

,Cédula,Tipo de Cédula,Sexo,Nombre,Paterno,Materno,Escuela,Título
0,2590660,C1,Maculino,JOSÉ LUIS,CRUZ,JARAMILLO,COLEGIO NACIONAL DE EDUCACIÓN PROFESIONAL TÉCNICA,PROFNAL. TEC. COMO QUÍMICO ESP. PROCESOS DE PR...
1,8666598,C1,Maculino,JOSE LUIS,CRUZ,JARAMILLO,INSTITUTO POLITÉCNICO NACIONAL,LICENCIATURA EN INGENIERÍA BIOTECNOLÓGICA
2,12146217,C1,Maculino,JOSE ALBERTO,MONTAN,LOPEZ,INSTITUTO TECNOLÓGICO Y DE ESTUDIOS SUPERIORES...,LICENCIATURA EN INGENIERO EN MECATRÓNICA
3,13574048,C1,Maculino,JOSE ALBERTO,MONTAN,LOPEZ,INSTITUTO TECNOLÓGICO Y DE ESTUDIOS SUPERIORES...,MAESTRÍA EN CIENCIAS COMPUTACIONALES
